In [28]:
!pip install xmltodict

In [2]:
import pandas as pd
import requests
from collections import defaultdict
import time
import json
import xml
import xml.etree.ElementTree as ET
import xmltodict
headers = {}
payload = {}

In [1]:
def default_value():
    return -1

df = pd.DataFrame(columns=['Name', 'Label', 'Lat', 'Long', 'Class'])

def get_addresses_of_poi(poi_type, radius):
    result_total = []
    for t in poi_type:
        params = {'placetype':t}
        api_url = 'http://localhost/geoloc/search?'
        result = []
        seen = defaultdict(default_value)
        stack = []
        url = api_url
        url += 'lat=' + str(47.70704242989272) + '&'
        url += 'lng=' + str(-122.2097948701073) + '&'
        url += 'radius=' + str(radius) + '&'
        for p in params.keys():
            url += p + '=' + params[p]
        response = requests.request("GET", url, headers=headers, data=payload)
        response = xmltodict.parse(response.content)
        stack.append([response['results']['result'][0]['name'], response['results']['result'][0]['label'], response['results']['result'][0]['lat'],
                    response['results']['result'][0]['lng'], response['results']['result'][0]['placeType']])
        while stack:
            url = api_url
            cur = stack[0]
            del stack[0]
            if seen[cur[0]] < 0:
                if len(result) % 1000 == 0:
                    print(len(result))
                seen[cur[0]] = 1
                result.append(cur)
                url += 'lat=' + str(cur[2]) + '&'
                url += 'lng=' + str(cur[3]) + '&'
                url += 'radius=' + str(radius) + '&'
                for p in params.keys():
                    url += p + '=' + params[p]
                try:
                    response = requests.request("GET", url, headers=headers, data=payload)
                    response = xmltodict.parse(response.content)
                    for r in response['results']['result']:
                        try:
                            if seen[r['name']] < 0:
                                stack.append([r['name'], r['label'], r['lat'], r['lng'], r['placeType']])
                        except:
                            pass
                except:
                    pass
            if len(result) > 18000:
                break
                
        result_total += result

    return result, stack

r, s = get_addresses_of_poi(['airport','factory', 'stadium'], 1000000)
new_df = pd.DataFrame(r, columns=['Name', 'Label', 'Lat', 'Long', 'Class'])
df = pd.concat([df, new_df])
df.to_csv('./GeoData.csv')

NameError: name 'pd' is not defined

In [ ]:
df